# Kapitel 1: Der Händedruck des Internets – Sockets & Protokolle

Willkommen zum ersten Kapitel unserer Reise in die Netzwerkprogrammierung. Bevor wir mit komfortablen Bibliotheken auf APIs zugreifen, steigen wir eine Ebene tiefer und schauen uns an, wie Computer auf der fundamentalsten Ebene miteinander "sprechen". Wir werden lernen, wie man einen digitalen "Händedruck" über das Internet durchführt und eine Nachricht manuell versendet.

## Das Fundament verstehen: Wie Computer sprechen

### 1.1 Sockets, IP-Adressen und Ports

Stellen Sie sich vor, Sie möchten einem Freund einen Brief schicken. Sie benötigen zwei Dinge: seine Wohnadresse und seinen Namen. Im Internet ist das ganz ähnlich:

  * **IP-Adresse:** Das ist die eindeutige Adresse eines Computers im Netzwerk (z.B. `142.250.185.99`). Es ist die "Wohnadresse" des Servers. Domainnamen wie `www.google.com` sind nur menschenfreundliche Aliase für diese Adressen.
  * **Port:** Ein Server bietet oft viele verschiedene Dienste an (Webseiten, E-Mails, Dateiübertragungen). Ein Port ist eine Nummer zwischen 1 und 65535, die einen bestimmten Dienst auf dem Server wie eine "Wohnungstür" adressiert.
      * **Port 80:** Der Standard-Port für HTTP (unverschlüsselte Webseiten).
      * **Port 443:** Der Standard-Port für HTTPS (verschlüsselte Webseiten).
  * **Socket:** Ein Socket ist die **Kombination aus IP-Adresse und Port**. Es ist der konkrete Endpunkt der Kommunikation, quasi die "genaue Lieferadresse inklusive Name am Briefkasten".

### 1.2 Protokolle: Die Regeln des Gesprächs

Ein Protokoll ist die Grammatik der Kommunikation. Damit Client und Server sich verstehen, müssen sie sich an dieselben Regeln halten.

  * **TCP (Transmission Control Protocol):** Dies ist ein **verbindungsorientiertes** Protokoll. Es stellt sicher, dass alle Datenpakete vollständig, fehlerfrei und in der richtigen Reihenfolge ankommen. Man kann es sich wie ein Einschreiben mit Sendungsverfolgung vorstellen.
  * **HTTP (Hypertext Transfer Protocol):** Das ist das Protokoll für das Web. Es läuft *auf* TCP und definiert, wie Anfragen (Requests) und Antworten (Responses) für Web-Inhalte aussehen müssen.

### 1.3 Die Anatomie einer rohen HTTP-Anfrage

Wenn wir Sockets manuell nutzen, müssen wir eine HTTP-Anfrage selbst als Text zusammenbauen. Dabei sind zwei Dinge extrem wichtig:

1.  **Bytes, nicht Strings:** Sockets senden und empfangen rohe **Bytes**. Wir müssen unseren Anfrage-String daher mit `.encode('utf-8')` in Bytes umwandeln und die Antwort mit `.decode('utf-8')` wieder in einen String zurückverwandeln.
2.  **Zeilenumbrüche:** Der HTTP-Standard schreibt vor, dass Zeilen mit einem "Carriage Return" gefolgt von einem "Line Feed" (`\r\n`) enden müssen. Eine leere Zeile (`\r\n\r\n`) signalisiert das Ende der Header und den Beginn des Bodys.

-----

## Die Theorie in Aktion: Unser erster manueller Web-Request

Die folgenden Schritte bauen aufeinander auf. Die Erklärungen stehen nun ausführlich vor dem jeweiligen Code-Block.

### Schritt 1: Import und Definition der Ziele

Zu Beginn importieren wir die `socket`-Bibliothek, die in Python standardmäßig enthalten ist, und legen unser Ziel fest. Wir definieren den Hostnamen als String und den Port als Integer.

```python
import socket

# Zielserver und Port definieren
target_host = "example.com"
target_port = 80 # HTTP-Standardport
```

### Schritt 2: Das Socket-Objekt erstellen

Nun erzeugen wir mit `socket.socket()` unser lokales Socket-Objekt. Wir müssen ihm zwei Konstanten übergeben, die seine "Familie" und seinen "Typ" festlegen:

  * `socket.AF_INET`: Dies legt die Adressfamilie fest. `AF_INET` steht für die Standard-IPv4-Adressierung, die wir für Domainnamen wie `example.com` verwenden.
  * `socket.SOCK_STREAM`: Dies legt den Socket-Typ fest. `SOCK_STREAM` steht für ein verbindungsorientiertes TCP-Protokoll. Es garantiert eine zuverlässige, geordnete Datenübertragung, wie ein kontinuierlicher Datenstrom.

**Was ist `client_socket` jetzt?**
Zu diesem Zeitpunkt ist das `client_socket`-Objekt nur ein ungebundener Endpunkt in unserem Betriebssystem. Es ist noch nicht mit dem Netzwerk verbunden und hat weder eine eigene noch eine Ziel-IP-Adresse zugewiesen bekommen. Es ist wie ein Telefonhörer, der noch nicht mit der Telefondose verbunden ist.

```python
# Erstellt ein TCP/IPv4 Socket-Objekt
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

print("Socket-Objekt wurde lokal erstellt.")
```

### Schritt 3: Die Verbindung herstellen

Mit der `.connect()`-Methode weisen wir unser Socket an, eine Verbindung zum Ziel aufzubauen. Diese Methode nimmt ein **Tupel** aus (`host`, `port`) entgegen. Intern löst Python nun den Hostnamen `example.com` zu einer IP-Adresse auf und führt den TCP-"Drei-Wege-Händedruck" aus, um eine stabile Verbindung zu etablieren.

**Was ist `client_socket` jetzt?**
Nach einem erfolgreichen `.connect()` ist unser Socket eine voll funktionsfähige Kommunikationsverbindung. Das Betriebssystem hat ihm eine lokale, kurzlebige Portnummer zugewiesen und ihn fest mit dem Endpunkt (der IP-Adresse von `example.com` und Port 80) verbunden. Man kann es sich so vorstellen, dass die Telefonverbindung nun steht und wir mit dem Sprechen beginnen können.

```python
# Verbindet den Socket mit dem Zielserver
client_socket.connect((target_host, target_port))

print(f"Verbindung zu {target_host}:{target_port} hergestellt.")
```

### Schritt 4: Die HTTP-Anfrage erstellen und senden

Jetzt kommt der manuelle Teil. Wir formulieren unsere HTTP-Anfrage als String.

**Die Bedeutung von `Connection: close`**
Moderne Webserver (HTTP/1.1) versuchen, eine Verbindung standardmäßig offenzuhalten (`Keep-Alive`), um weitere Anfragen über denselben Socket zu ermöglichen. Unser einfaches Skript ist darauf aber nicht ausgelegt; es wüsste nicht, wann die Antwort des Servers komplett ist, und würde ewig auf weitere Daten warten. Der Header `Connection: close\r\n` ist eine explizite Anweisung an den Server: "Bitte sende mir deine Antwort und schließe die Verbindung danach sofort." Das gibt unserem Skript ein klares Signal (eine leere Antwort), wann die Übertragung beendet ist.

```python
# Eine minimale HTTP-GET-Anfrage als String
# Wichtig: \r\n für Zeilenenden und eine leere Zeile zum Schluss.
request_string = (
    f"GET / HTTP/1.1\r\n"
    f"Host: {target_host}\r\n"
    f"Connection: close\r\n"
    f"\r\n"
)

# Sende die Anfrage als Bytes
client_socket.send(request_string.encode('utf-8'))

print("HTTP-Anfrage gesendet.")
```

### Schritt 5: Die Antwort empfangen und ausgeben

Da wir nicht wissen, wie groß die Antwort sein wird, empfangen wir sie in einer Schleife in kleinen Stücken (Chunks) von z.B. 4096 Bytes. Die Schleife läuft so lange, bis `client_socket.recv()` einen leeren Byte-String (`b''`) zurückgibt – das ist das Signal, das der Server (dank `Connection: close`) die Verbindung beendet hat. Die empfangenen Bytes hängen wir an eine Liste an und setzen sie am Ende zusammen.

```python
# Eine Liste, um die empfangenen Daten-Chunks zu sammeln
received_data = []

while True:
    # Empfange Daten in Chunks von 4096 Bytes
    chunk = client_socket.recv(4096)
    if not chunk:
        # Wenn ein leerer Chunk empfangen wird, hat der Server die Verbindung geschlossen.
        break
    received_data.append(chunk)

# Schließe die Verbindung auf unserer Seite
client_socket.close()

# Setze alle Chunks zu einem einzigen Byte-String zusammen
full_response_bytes = b"".join(received_data)

# Dekodiere den Byte-String und gib ihn aus
print("--- ANTWORT VOM SERVER ---")
print(full_response_bytes.decode('utf-8'))
print("\n--- Verbindung wurde geschlossen ---")
```

-----

## Das Projekt-Fundament: Manuelle Anfrage an die Buch-API

Wir wenden den exakt gleichen, robusten Prozess nun auf den Server der "Open Library" API an, um zu sehen, wie die rohe Antwort von dort aussieht.

```python
import socket

api_host = "openlibrary.org"
api_port = 80

api_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
api_socket.connect((api_host, api_port))

# Wir verwenden auch hier "Connection: close" für eine saubere Trennung.
request = f"GET / HTTP/1.1\r\nHost: {api_host}\r\nConnection: close\r\n\r\n"
api_socket.send(request.encode('utf-8'))

# Robuster Empfang in einer Schleife
response_parts = []
while True:
    part = api_socket.recv(4096)
    if not part:
        break
    response_parts.append(part)

api_socket.close()

# Antwort zusammensetzen und ausgeben
full_response = b"".join(response_parts)
print(full_response.decode('utf-8', errors='ignore'))
```

-----

### Deine Werkstatt: Übungen zu Kapitel 1

# Übungs-Set 1: Erste Schritte mit Sockets

**Ziel:** In diesen Übungen machen Sie sich mit der `socket`-Bibliothek vertraut und verstehen die Grundlagen der direkten Netzwerkkommunikation.

### Aufgabe 1: Eigene Verbindung zu `example.com`

Schreiben Sie ein komplettes Skript, das alle 5 Schritte aus dem Beispiel (Setup, Erstellen, Verbinden, Senden, Empfangen/Schließen) kombiniert, um den Inhalt der Seite `http://example.com/` abzurufen und in der Konsole auszugeben.

### Aufgabe 2: Ein anderer Host

Modifizieren Sie Ihr Skript aus Aufgabe 1. Versuchen Sie, sich mit `www.google.com` auf Port 80 zu verbinden. Analysieren Sie die Antwort. Was sagt die "Status Line"? Was steht im `Location`-Header?

### Aufgabe 3: Ein anderer Port

Versuchen Sie, sich mit `example.com` auf Port 81 zu verbinden. Was passiert? Fangen Sie den Fehler mit einem `try-except`-Block ab und geben Sie eine benutzerfreundliche Fehlermeldung aus, z.B. "Verbindung fehlgeschlagen. Ist der Port korrekt?".

### Aufgabe 4: Manuelle Header-Analyse

Nehmen Sie die vollständige Antwort aus Aufgabe 1. Schreiben Sie Code, der die Antwort in zwei Teile teilt: den Header-Block und den Body (HTML-Inhalt). Der Trenner ist immer die erste leere Zeile (`\r\n\r\n`). Geben Sie nur den Body aus.

### Aufgabe 5: **Schüler-Projekt: Anfrage an JSONPlaceholder**

Schreiben Sie ein Skript, das eine manuelle HTTP-Anfrage an den Host `jsonplaceholder.typicode.com` (Port 80) sendet.

  * Fordern Sie den Pfad `/posts/1` an.
  * Geben Sie die vollständige Antwort (Header und Body) aus. Sie sollten eine JSON-Struktur im Body sehen.

### Aufgabe 6 (Challenge): Hostname zu IP-Adresse

Das `socket`-Modul hat eine Funktion `gethostbyname()`. Schreiben Sie ein Skript, das einen Domainnamen (z.B. `"www.python.org"`) entgegennimmt und dessen IP-Adresse ausgibt.

### Aufgabe 7 (Challenge): Ein minimaler Port-Scanner

Schreiben Sie ein Skript, das versucht, sich mit einem Host (z.B. `example.com`) nacheinander auf den Ports 79, 80 und 81 zu verbinden. Geben Sie für jeden Port aus, ob die Verbindung erfolgreich war ("Port X ist offen") oder fehlgeschlagen ist ("Port X ist geschlossen").